In [1]:
import collections
import numpy as np
import torch
import os

from data_loader import load_data
from datetime import datetime
from functools import partial
from inference import predict
from models import AlexNet
from svcca import cca_core
from scipy.spatial import distance
from scipy import stats


def hook_fn(name, activations, module, input, output):
    activations[name].append(output.half().to("cpu"))


def set_hooks(model, activations):
    # names = ["conv1", "conv2", "conv3", "conv4", "conv5", "dense1", "dense2", "dense3"]
    # layers = [model.features[0], model.features[3], model.features[6], model.features[8], model.features[10], model.classifier[1], model.classifier[4], model.classifier[6]]
    
    names = ["conv1"]
    layers = [model.features[0]]

    for name, layer in zip(names, layers):
        layer.register_forward_hook(partial(hook_fn, name, activations))


def get_activations(net, data_loader, device):
    model = net.model
    model.to(device)

    activations = collections.defaultdict(list)

    set_hooks(model, activations)

    predict(model, data_loader, device)
    torch.cuda.empty_cache()

    for _ in activations:
        activations[_] = torch.cat(activations[_])

    return activations

In [2]:
def partial_save(activations, key, label):
    directory = 'data/activations/tmp'
    
    t = activations[key].transpose(0, 1).reshape(activations[key].shape[1], -1).numpy()
    
    for i, unit in enumerate(t):
        torch.save(unit, f"{directory}/{label}_{key}_{i}.pth")
    
    return t.shape[0]

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 2400
num_workers = 6
data_loader = load_data(split="val", batch_size=batch_size, num_workers=num_workers)

root = "data/model-weights/pytorch-vision-classification/run_1"

final_net = AlexNet()
final_net.set_weights(f"{root}/model_89.pth")

A_f = get_activations(final_net, data_loader, device)
partial_save(A_f, 'conv1', "A_f")

del A_f

scores = []
pearson_scores = []

for num in range(0,90):
    print(f"model_{num}")

    net = AlexNet()
    net.set_weights(f"{root}/model_{num}.pth")

    A_i = get_activations(net, data_loader, device)
    length = partial_save(A_i, 'conv1', "A_i")
    
    del A_i
    
    for i in range(length):
        single_A_f = torch.load(f'data/activations/tmp/A_f_conv1_{i}.pth')
        single_A_i = torch.load(f'data/activations/tmp/A_i_conv1_{i}.pth')
    
        cca = cca_core.get_cca_similarity(single_A_f.reshape(1, -1), single_A_i.reshape(1, -1), epsilon=1e-6, verbose=False)
        score = np.mean(cca["cca_coef1"])
        pearson = stats.pearsonr(single_A_f, single_A_i)[0]
        
        scores.append(score)
        pearson_scores.append(pearson)
    
        np.save('conv1-scores.npy', scores)
        np.save('conv1-pearson_scores.npy', pearson_scores)
        
        os.remove(f'data/activations/tmp/A_i_conv1_{i}.pth') 

model_0
model_1


KeyboardInterrupt: 

In [ ]:
#     for layer in ["conv1"]:
#         data_f = A_f[layer].transpose(0, 1).reshape(A_f[layer].shape[1], -1).cpu().detach().numpy().T
#         data_i = A_i[layer].transpose(0, 1).reshape(A_i[layer].shape[1], -1).cpu().detach().numpy().T

#         for idx in range(len(data_i)):
#             cca = cca_core.get_cca_similarity(data_f[[idx]], data_i[[idx]], epsilon=1e-6, verbose=False)
#             score = np.mean(cca["cca_coef1"])
            
#             pearson = stats.pearsonr(data_f[idx], data_i[idx])[0]
            
#             scores[layer].append(score)
#             pearson_scores[layer].append(pearson)
            
#     np.save('conv1-scores.npy', dict(scores))
#     np.save('conv1-pearson_scores.npy', dict(pearson_scores))

In [ ]:
np.load